# Analisis Pasien Rawat Inap RS
### Deskripsi Dataset
Dataset yang digunakan adalah dataset Hospital Inpatient Discharges (SPARCS De-Identified): 2016 yang didapatkan dari open data kesehatan New York State. Berikut ini adalah beberapa analisis yang dilakukan untuk mendapatkan insight dari dataset tersebut.

Sistem Kooperatif Penelitian dan Perencanaan Seluruh Negara Bagian (SPARCS) Rawat Inap diidentifikasi File berisi detail tingkat debit pada karakteristik pasien, diagnosis, perawatan, layanan, dan biaya. File data ini berisi detail level catatan dasar untuk debit. File data yang tidak diidentifikasi tidak berisi data yang dilindungi informasi kesehatan (PHI) di bawah HIPAA. Informasi kesehatan tidak dapat diidentifikasi secara individual; semua elemen data yang dianggap dapat diidentifikasi telah dihapus. Misalnya, pengidentifikasi langsung mengenai tanggal menghapus bagian hari dan bulan dari tanggal tersebut.

In [1]:
#import library
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as f
from pyspark.sql.types import DateType
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import json
%matplotlib inline

import plotly
import plotly.plotly as py
#plotly.tools.set_credentials_file(username='akbar0102', api_key='HhbZVjpsfob4yDF1aGpd')
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
#buat objek spark
spark = SparkSession.builder.appName('nyc_health').getOrCreate()
sqlContext = SQLContext(spark)

In [3]:
#read data csv
rawat = spark.read.csv('raw_data/NYSDOH_Hospital_Inpatient_Discharges_SPARCS_De-Identified_2016.csv', inferSchema=True, header=True)

In [4]:
#lihat jumlah kolom dan rekord
print('Data Rawat Inap',rawat.count(), len(rawat.columns))

Data Rawat Inap 2343429 38


In [5]:
rawat.printSchema()

root
 |-- Hospital Service Area: string (nullable = true)
 |-- Hospital County: string (nullable = true)
 |-- Operating Certificate Number: integer (nullable = true)
 |-- Permanent Facility Id: integer (nullable = true)
 |-- Facility Name: string (nullable = true)
 |-- Age Group: string (nullable = true)
 |-- Zip Code - 3 digits: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Ethnicity: string (nullable = true)
 |-- Length of Stay: string (nullable = true)
 |-- Type of Admission: string (nullable = true)
 |-- Patient Disposition: string (nullable = true)
 |-- Discharge Year: integer (nullable = true)
 |-- CCS Diagnosis Code: integer (nullable = true)
 |-- CCS Diagnosis Description: string (nullable = true)
 |-- CCS Procedure Code: integer (nullable = true)
 |-- CCS Procedure Description: string (nullable = true)
 |-- APR DRG Code: integer (nullable = true)
 |-- APR DRG Description: string (nullable = true)
 |-- APR MDC Code: inte

### <font color=blue>*Analisis demografi pasien rawat inap* </font>

#### *Bagaimana demografi karakteristik usia pasien rawat inap?* <br>
Pasien paling banyak yang menjalani rawat inap adalah pasien dengan rentang usia 50-69 tahun kemudian dengan perbedaan sedikit ada pada rentang usia 70-lebih tua, sementara remaja yang berusia 18-29 tahun yang paling sedikit menjalani rawat inap.

In [82]:
# rentang usia pasien bagi laki-laki dan perempuan
df = rawat.groupBy('Age Group').count().sort('Age Group')
#df = df.toPandas()

In [7]:
trace0 = go.Bar(
    x=df['Age Group'],
    y=df['count']
)

data = [trace0]
layout = go.Layout(
    title='Rentang Usia Pasien 2016',
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='age-group-patient')

In [84]:
# export dataframe ke json
results = df.toJSON().map(lambda j: json.loads(j)).collect()
# write file json ke lokal
with open('pasien_usia_2016.json', 'w') as f:
     json.dump(results, f)

#### *Bagaimana demografi karakteristik jenis kelamin pasien rawat inap?*
Sebagian besar pasien rawat inap adalah wanita sebesar 55%, dibanding pria hanya sekitar 44,5%. Jumlah pasien wanita terbanyak karena salah satu kasus yang sering dilayani di rumah sakit adalah proses persalinan.

In [85]:
# pasien berdasarkan jenis kelamin
gender = rawat.groupBy('Gender').count()
#gender = gender.toPandas()

In [8]:
trace0 = go.Pie(
    labels=gender['Gender'],
    values=gender['count']
)

data = [trace0]
layout = go.Layout(
    title='Jenis Kelamin Pasien 2016',
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='age-group-patient')

In [86]:
# export dataframe ke json
results = gender.toJSON().map(lambda j: json.loads(j)).collect()
# write file json ke lokal
with open('pasien_gender.json', 'w') as f:
     json.dump(results, f)

#### *Bagaimana demografi karakteristik ras pasien rawat inap?*

In [9]:
# ras pasien
ras = rawat.groupBy('Race').count()
ras = ras.toPandas()

In [10]:
trace0 = go.Pie(
    labels=ras['Race'],
    values=ras['count']
)

data = [trace0]
layout = go.Layout(
    title='Ras Pasien 2016',
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='race-group-patient')

In [11]:
# jenis kelamin untuk setiap ras
frace = rawat.filter(rawat['Gender']=='F').groupBy('Race').count()
mrace = rawat.filter(rawat['Gender']=='M').groupBy('Race').count()
frace = frace.toPandas()
mrace = mrace.toPandas()

In [12]:
fig = {
  "data": [
    {
      "values": mrace['count'],
      "labels": mrace['Race'],
      "domain": {"x": [0, .48]},
      "name": "Male",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    },
    {
      "values": frace['count'],
      "labels": frace['Race'],
      "textposition":"inside",
      "domain": {"x": [.52, 1]},
      "name": "Female",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    }],
  "layout": {
        "title":"Ras Pasien Jenis Kelamin 2016",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Male",
                "x": 0.20,
                "y": 0.5
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Female",
                "x": 0.8,
                "y": 0.5
            }
        ]
    }
}
iplot(fig, filename='donut-patient-race')

In [20]:
# rentang usia berdasarkan jenis kelamin
male = rawat.filter(rawat['Gender']=='M').groupBy('Age Group','Gender').count()
female = rawat.filter(rawat['Gender']=='F').groupBy('Age Group', 'Gender').count()
#male = male.toPandas()
#female = female.toPandas()

In [22]:
df_concat = male.union(female)

In [24]:
df_concat.show()

+-----------+------+------+
|  Age Group|Gender| count|
+-----------+------+------+
|    0 to 17|     M|180107|
|   18 to 29|     M| 67581|
|70 or Older|     M|278745|
|   50 to 69|     M|348822|
|   30 to 49|     M|166446|
|   30 to 49|     F|288479|
|    0 to 17|     F|167496|
|70 or Older|     F|363936|
|   18 to 29|     F|176062|
|   50 to 69|     F|305698|
+-----------+------+------+



In [14]:
trace1 = go.Bar(
    x=male['Age Group'],
    y=male['count'],
    name='Male'
)
trace2 = go.Bar(
    x=female['Age Group'],
    y=female['count'],
    name='Female'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack',
    title='Jenis Kelamin Rentang Usia Pasien'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='sex-age-patient')

In [23]:
# export dataframe ke json
results = df_concat.toJSON().map(lambda j: json.loads(j)).collect()
# write file json ke lokal
with open('pasien-jenis-kelamin-usia.json', 'w') as f:
     json.dump(results, f)

### <font color=blue>*Analisis alasan penyebab pasien dirawat inap dan perawatannya* </font>

#### *Pasien dengan tipe penerimaan apa yang paling banyak diterima oleh rumah sakit?*
Pasien rawat inap paling banyak masuk ke rumah sakit dengan kondisi darurat, yang membutuhkan penanganan segera.

In [88]:
# jenis penerimaan pasien rawat inap
admission = rawat.groupBy('Type of Admission').count().sort('count',ascending=False)

In [89]:
# export dataframe ke json
results = admission.toJSON().map(lambda j: json.loads(j)).collect()
# write file json ke lokal
with open('pasien_admission.json', 'w') as f:
     json.dump(results, f)

#### *Apa saja kondisi klinis yang mengakibatkan rawat inap darurat?*
Pasien yang paling banyak membutuhkan rawat inap darurat adalah penyakit keracunan darah, selanjutnya gagal jantung. Beberapa penyakit tersebut merupakan penyakit yang harus segera ditangani untuk menyelamatkan pasien.

In [6]:
# cara pasien dirawat di fasilitas perawatan kesehatan, menggunakan fasilitas dengan kategori emergency
# berdasarkan penyakitnya 
# pasien dengan penyakit/diagnostik apa yang perlu fasilitas emergency atau harus segera dilakukan penanganan?
emergensi = rawat.filter(rawat['Type of Admission']=='Emergency').groupBy('APR DRG Description').count().sort('count',ascending=False).limit(20)

In [9]:
emergensi = emergensi.toPandas()

In [10]:
emergensi.to_json(path_or_buf='pasien_admission_emergensi.json',orient='table')

In [91]:
# export dataframe ke json
results = emergensi.toJSON().map(lambda j: json.loads(j)).collect()
# write file json ke lokal
with open('pasien_admission_emergency.json', 'w') as f:
     json.dump(results, f)

#### *Alasan apa yang paling sering menyebabkan pasien dirawat di rumah sakit?*

In [13]:
ccs_diagnostik = rawat.groupBy('CCS Diagnosis Description').count().sort('count',ascending=False).limit(10)

In [14]:
# export dataframe ke json
results = ccs_diagnostik.toJSON().map(lambda j: json.loads(j)).collect()
# write file json ke lokal
with open('pasien_diagnostik.json', 'w') as f:
     json.dump(results, f)

#### *Prosedur utama apa yang paling sering dilakukan?*

In [20]:
rawat.groupBy('CCS Procedure Description').count().sort('count',ascending=False).show()

+-------------------------+------+
|CCS Procedure Description| count|
+-------------------------+------+
|                  NO PROC|696951|
|     OTHER THERAPEUTIC...|154778|
|     OT PRCS TO ASSIST...|110287|
|         CESAREAN SECTION| 72617|
|     RESP INTUB/MECH V...| 71616|
|     PROPHYLACTIC VAC/...| 67692|
|     ALCO/DRUG REHAB/D...| 59688|
|     PSYCHO/PSYCHI EVA...| 50991|
|             CIRCUMCISION| 46574|
|        BLOOD TRANSFUSION| 40130|
|     OPHTHALM-/OT-OLOG...| 40124|
|        ARTHROPLASTY KNEE| 39347|
|     HIP REPLACEMENT,T...| 32645|
|     DX CARDIAC CATHET...| 31025|
|      DX ULTRASOUND HEART| 26713|
|     OT VASC CATH; NOT...| 26590|
|             HEMODIALYSIS| 25652|
|     PERC TRANSLUM COR...| 23599|
|            SPINAL FUSION| 23414|
|     UP GASTRO ENDOSC/...| 19383|
+-------------------------+------+
only showing top 20 rows



#### *Saat pasien pulang, perawatan apa yang paling banyak dilakukan selanjutnya?*
Setelah menjalani proses rawat inap, perawatan lanjutan bagi pasien paling banyak adalah perawatan lanjutan di rumah secara mandiri. Baik itu menggunakan perawat atau tidak.

In [16]:
# tujuan atau status pasien setelah keluar dari rs untuk melanjutkan perawatannya
# penyakit tertentu misalnya(jantung) paling banyak perawatan apa yang dilakukan setelah pasien pulang?
disposisi=rawat.groupBy('Patient Disposition').count().sort('count',ascending=False).limit(10)

In [17]:
# export dataframe ke json
results = disposisi.toJSON().map(lambda j: json.loads(j)).collect()
# write file json ke lokal
with open('pasien_disposisi.json', 'w') as f:
     json.dump(results, f)

#### *Metode pembayaran apa yang paling sering digunakan oleh pasien?*

In [14]:
# metode pembayaran apa yang sering atau paling banyak digunakan oleh pasien?
# sebagai informasi penting untuk penyedia jasa pembayaran, misalnya penyakit jantung lebih sering menggunakan pembayaran tertentu.
# sehingga bisa untuk mengambil keputusan
rawat.groupBy('Payment Typology 1').count().sort('count',ascending=False).show()

+--------------------+------+
|  Payment Typology 1| count|
+--------------------+------+
|            Medicare|884212|
|            Medicaid|723774|
|Private Health In...|333581|
|Blue Cross/Blue S...|271620|
|            Self-Pay| 54795|
| Miscellaneous/Other| 27758|
|Managed Care, Uns...| 20288|
|Federal/State/Loc...| 16576|
|             Unknown|  6896|
|Department of Cor...|  3929|
+--------------------+------+



### <font color=blue>*Analisis penyakit tertentu* </font>

#### *Persebaran penyakit asma pada rentang usia berapa terbanyak?*


In [13]:
# Semua Kategori Diagnostik Utama Pasien yang Disempurnakan, diagnostik penyakit tertentu
# Asma
# penyakit apa yang paling banyak diagnostiknya menuruta rentang usia pasien?
rawat.filter(rawat['CCS Diagnosis Code'] == 128).select('APR DRG Description', 'APR DRG Code', 'Age Group')\
    .groupBy('APR DRG Description', 'APR DRG Code','Age Group').count().sort('count',ascending=False).limit(5).show()

+-------------------+------------+-----------+-----+
|APR DRG Description|APR DRG Code|  Age Group|count|
+-------------------+------------+-----------+-----+
|             Asthma|         141|    0 to 17| 9814|
|             Asthma|         141|   50 to 69| 4414|
|             Asthma|         141|   30 to 49| 3452|
|             Asthma|         141|70 or Older| 1957|
|             Asthma|         141|   18 to 29| 1685|
+-------------------+------------+-----------+-----+



#### *Kondisi kesehatan Sistem Peredaran Darah*
Diagnostik kesehatan peredaran darah paling banyak adalah gagal jantung, kemudian beberapa penyakit yang berkaitan dengan jantung.

In [16]:
rawat.filter((rawat['APR MDC Code'] == 5) & (rawat['APR DRG Code'] == 194)).select('APR DRG Description','Age Group')\
    .groupBy('APR DRG Description','Age Group').count().sort('count',ascending=False).show()

+-------------------+-----------+-----+
|APR DRG Description|  Age Group|count|
+-------------------+-----------+-----+
|      Heart failure|70 or Older|37426|
|      Heart failure|   50 to 69|16237|
|      Heart failure|   30 to 49| 2784|
|      Heart failure|   18 to 29|  194|
|      Heart failure|    0 to 17|   67|
+-------------------+-----------+-----+



### <font color=blue>*Analisis kewilayahan (daerah) rumah sakit* </font>

#### *Ranking daerah berdasarkan pasien yang keluar dari rumah sakit?*

In [25]:
rawat.groupBy('Hospital County').count().sort('count',ascending=False).show(10)

+---------------+------+
|Hospital County| count|
+---------------+------+
|      Manhattan|394810|
|          Kings|239213|
|         Queens|200711|
|          Bronx|183279|
|         Nassau|182237|
|        Suffolk|160835|
|           Erie|123687|
|    Westchester|118970|
|         Monroe|110162|
|       Onondaga| 79794|
+---------------+------+
only showing top 10 rows



In [26]:
rawat.groupBy('Hospital Service Area','Hospital County').count().sort('count',ascending=False).show(5)

+---------------------+---------------+------+
|Hospital Service Area|Hospital County| count|
+---------------------+---------------+------+
|        New York City|      Manhattan|394810|
|        New York City|          Kings|239213|
|        New York City|         Queens|200711|
|        New York City|          Bronx|183279|
|          Long Island|         Nassau|182237|
+---------------------+---------------+------+
only showing top 5 rows



#### *Rawat inap berdasarkan ukuran rumah sakit (jumlah kasur dan service yang tersedia)?*

In [13]:
rawat.groupBy('Facility Name', 'Permanent Facility Id').count().sort('count',ascending=False).show()

+--------------------+---------------------+-----+
|       Facility Name|Permanent Facility Id|count|
+--------------------+---------------------+-----+
|Mount Sinai Hospital|                 1456|56201|
|North Shore Unive...|                  541|48909|
|New York Presbyte...|                 1464|47686|
|Montefiore Medica...|                 1169|42669|
|Long Island Jewis...|                 1630|42390|
|Strong Memorial H...|                  413|41671|
|Maimonides Medica...|                 1305|41525|
|New York Presbyte...|                 1458|40577|
|Albany Medical Ce...|                    1|38988|
|NewYork-Presbyter...|                 1306|38506|
|Winthrop-Universi...|                  511|35830|
| University Hospital|                  245|33937|
|NYU Hospitals Center|                 1463|32902|
|Rochester General...|                  411|31829|
|New York Hospital...|                 1637|31436|
| Lenox Hill Hospital|                 1450|31209|
|Staten Island Uni...|         

In [79]:
fasilitas.select(f.col("Facility Name").alias("Facility"), f.col("sum(Measure Value)").alias("Bed")).show()

+--------------------+------+
|            Facility|   Bed|
+--------------------+------+
|Mount Sinai Hospital|1134.0|
|Long Island Jewis...|1025.0|
|New York Presbyte...|1022.0|
|Bellevue Hospital...| 912.0|
|New York Presbyte...| 862.0|
|Strong Memorial H...| 846.0|
|NYU Langone Hospi...| 844.0|
|Hebrew Home for t...| 843.0|
|Montefiore Medica...| 816.0|
|Coler Rehabilitat...| 815.0|
|The Plaza Rehab a...| 744.0|
|North Shore Unive...| 738.0|
|Kings Harbor Mult...| 720.0|
|Albany Medical Ce...| 716.0|
|Maimonides Medica...| 711.0|
|Isabella Geriatri...| 705.0|
|Mount Sinai Beth ...| 701.0|
|Westchester Medic...| 652.0|
|Kings County Hosp...| 639.0|
| Lenox Hill Hospital| 632.0|
+--------------------+------+
only showing top 20 rows



In [ ]:
# hospital bed
# large > 100
# small <= 100

#### *Pasien rawat inap berdasarkan lokasi rumah sakit?*

In [12]:
# penyakit apa yang paling banyak di diagnostik menurut county?
# bisa menjadi masukan kepada pemerintah setempat, untuk meningkatkan fasilitas atau melakukan pencegahan tertentu
# dalam mengambil keputusan terkait penyakit atau diagnostik yang paling banyak
rawat.filter(rawat['CCS Diagnosis Description']!='Liveborn').groupBy('Hospital County','CCS Diagnosis Description').count().sort('count',ascending=False).limit(20).show()

+---------------+-------------------------+-----+
|Hospital County|CCS Diagnosis Description|count|
+---------------+-------------------------+-----+
|      Manhattan|           Osteoarthritis|17219|
|      Manhattan|     Septicemia (excep...|11894|
|         Queens|     Septicemia (excep...|11088|
|          Kings|     Septicemia (excep...|10684|
|         Nassau|     Septicemia (excep...| 9517|
|      Manhattan|     Complication of d...| 9299|
|        Suffolk|     Septicemia (excep...| 8848|
|      Manhattan|     Spondylosis; inte...| 8261|
|      Manhattan|     Schizophrenia and...| 8052|
|      Manhattan|     Other complicatio...| 7775|
|          Kings|     Schizophrenia and...| 7597|
|      Manhattan|     Alcohol-related d...| 7591|
|      Manhattan|     Coronary atherosc...| 7519|
|      Manhattan|     Complications of ...| 7403|
|      Manhattan|           Mood disorders| 7264|
|      Manhattan|      Prolonged pregnancy| 6903|
|         Monroe|     Septicemia (excep...| 6709|


In [10]:
# rata-rata lama menginap pasien yang terdiagnosis penyakit tertentu
los = rawat.groupBy('Length of Stay','APR DRG Description').count().sort('Length of Stay',ascending=False)

In [11]:
los.show(5)

+--------------+--------------------+-----+
|Length of Stay| APR DRG Description|count|
+--------------+--------------------+-----+
|            99|Skin graft, excep...|    1|
|            99|Infectious & para...|    3|
|            99|Neonate birthwt 5...|    5|
|            99|Female reproducti...|    1|
|            99|Septicemia & diss...|    1|
+--------------+--------------------+-----+
only showing top 5 rows



In [23]:
# menghapus simbol dollar
rawat.select("Total Charges", f.regexp_replace(f.col("Total Charges"), "[\$#,]", "").alias("replaced")).show()

+-------------+--------+
|Total Charges|replaced|
+-------------+--------+
|     3,913.23| 3913.23|
|     3,597.10| 3597.10|
|     9,149.22| 9149.22|
|     5,880.61| 5880.61|
|     7,019.76| 7019.76|
|     2,258.92| 2258.92|
|     3,211.73| 3211.73|
|     4,010.26| 4010.26|
|     4,049.92| 4049.92|
|     4,875.97| 4875.97|
|     6,066.93| 6066.93|
|     2,168.93| 2168.93|
|     3,549.64| 3549.64|
|     3,665.16| 3665.16|
|     4,734.03| 4734.03|
|     5,525.80| 5525.80|
|     1,702.20| 1702.20|
|     2,424.34| 2424.34|
|     2,030.05| 2030.05|
|     2,412.45| 2412.45|
+-------------+--------+
only showing top 20 rows



### Analisis Informasi Umum Fasilitas Kesehatan
Dataset ini berisi lokasi Pasal 28, Pasal 36 dan Pasal 40 fasilitas dan program layanan kesehatan dari Sistem Informasi Fasilitas Kesehatan (HFIS). Fasilitas Pasal 28 adalah rumah sakit, panti jompo, dan pusat perawatan diagnostik. Pasal 36 fasilitas adalah agen perawatan kesehatan rumah bersertifikat dan program perawatan kesehatan rumah jangka panjang. Fasilitas Pasal 40 adalah hospis. Dataset saat ini hanya berisi lokasi rumah sakit dan klinik ekstensi rumah sakit. Data untuk jenis fasilitas yang tersisa akan ditambahkan di masa mendatang.

In [ ]:
# Health Facility General Information
# Health Facility Certification Information

In [6]:
health_general = spark.read.csv('raw_data/Health_Facility_General_Information.csv', inferSchema=True, header=True)
health_cert = spark.read.csv('raw_data/Health_Facility_Certification_Information.csv', inferSchema=True, header=True)

In [14]:
health_general.printSchema()

root
 |-- Facility ID: integer (nullable = true)
 |-- Facility Name: string (nullable = true)
 |-- Short Description: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Facility Open Date: string (nullable = true)
 |-- Facility Address 1: string (nullable = true)
 |-- Facility Address 2: string (nullable = true)
 |-- Facility City: string (nullable = true)
 |-- Facility State: string (nullable = true)
 |-- Facility Zip Code: string (nullable = true)
 |-- Facility Phone Number: long (nullable = true)
 |-- Facility Fax Number: long (nullable = true)
 |-- Facility Website: string (nullable = true)
 |-- Facility County Code: integer (nullable = true)
 |-- Facility County: string (nullable = true)
 |-- Regional Office ID: integer (nullable = true)
 |-- Regional Office: string (nullable = true)
 |-- Main Site Name: string (nullable = true)
 |-- Main Site Facility ID: integer (nullable = true)
 |-- Operating Certificate Number: string (nullable = true)
 |-- Operator Name

In [16]:
health_cert.printSchema()

root
 |-- Facility ID: integer (nullable = true)
 |-- Facility Name: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Short Description: string (nullable = true)
 |-- Attribute Type: string (nullable = true)
 |-- Attribute Value: string (nullable = true)
 |-- Measure Value: string (nullable = true)
 |-- Sub Type: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Regional Office: string (nullable = true)
 |-- Effective Date: string (nullable = true)



In [7]:
changedTypedf = health_cert.withColumn("Measure Value", f.col("Measure Value").cast("double"))

In [8]:
# jumlah kasur yang dimiliki semua fasilitas kesehatan seluruh NY
fasilitas = changedTypedf.filter(changedTypedf['Attribute Type']=='Bed').groupBy('Facility Name', 'Facility ID').sum('Measure Value')\
    .sort('sum(Measure Value)', ascending=False)
fasilitas.registerTempTable('fasilitas')

In [9]:
tes = rawat.select('Facility Name', 'Permanent Facility Id').distinct()

In [10]:
tes.filter(tes['Permanent Facility Id'] == 1).show()

+--------------------+---------------------+
|       Facility Name|Permanent Facility Id|
+--------------------+---------------------+
|Albany Medical Ce...|                    1|
+--------------------+---------------------+



In [11]:
fasilitas.select(f.col("Facility Name").alias("Facility"), f.col("sum(Measure Value)").alias("Bed")).show()

+--------------------+------+
|            Facility|   Bed|
+--------------------+------+
|Mount Sinai Hospital|1134.0|
|Long Island Jewis...|1025.0|
|New York Presbyte...|1022.0|
|Bellevue Hospital...| 912.0|
|New York Presbyte...| 862.0|
|Strong Memorial H...| 846.0|
|NYU Langone Hospi...| 844.0|
|Hebrew Home for t...| 843.0|
|Montefiore Medica...| 816.0|
|Coler Rehabilitat...| 815.0|
|The Plaza Rehab a...| 744.0|
|North Shore Unive...| 738.0|
|Kings Harbor Mult...| 720.0|
|Albany Medical Ce...| 716.0|
|Maimonides Medica...| 711.0|
|Isabella Geriatri...| 705.0|
|Mount Sinai Beth ...| 701.0|
|Westchester Medic...| 652.0|
|Kings County Hosp...| 639.0|
| Lenox Hill Hospital| 632.0|
+--------------------+------+
only showing top 20 rows



In [34]:
# jumlah kasur yang dimiliki semua fasilitas kesehatan rumah sakit seluruh NY
changedTypedf.filter((changedTypedf['Attribute Type']=='Bed') & (changedTypedf['Description']=='Hospital'))\
    .groupBy('Facility Name').sum('Measure Value')\
    .sort('Facility Name').show()

+--------------------+------------------+
|       Facility Name|sum(Measure Value)|
+--------------------+------------------+
|Adirondack Medica...|              95.0|
|Albany Medical Ce...|              18.0|
|Albany Medical Ce...|             716.0|
|Albany Memorial H...|             165.0|
|Alice Hyde Medica...|              76.0|
|Arnot Ogden Medic...|             266.0|
|Auburn Community ...|              99.0|
|Aurelia Osborn Fo...|              53.0|
|Bellevue Hospital...|             912.0|
|Bertrand Chaffee ...|              24.0|
|Blythedale Childr...|              86.0|
|Bon Secours Commu...|             122.0|
|BronxCare Hospita...|             585.0|
|Brookdale Hospita...|             530.0|
|Brooklyn Hospital...|             464.0|
|Brooks-TLC Hospit...|             130.0|
|Buffalo General M...|             484.0|
|    Calvary Hospital|              25.0|
|Calvary Hospital Inc|             200.0|
|Canton-Potsdam Ho...|              94.0|
+--------------------+------------

In [27]:
# jumlah fasilitas kasur yang dimiliki per daerah di New York State
changedTypedf.filter(changedTypedf['Attribute Type']=='Bed').groupBy('County').sum('Measure Value')\
    .sort('County').show()

+-----------+------------------+
|     County|sum(Measure Value)|
+-----------+------------------+
|     Albany|            3226.0|
|   Allegany|             437.0|
|      Bronx|           15314.0|
|     Broome|            2354.0|
|Cattaraugus|             738.0|
|     Cayuga|             611.0|
| Chautauqua|            1375.0|
|    Chemung|            1101.0|
|   Chenango|             622.0|
|    Clinton|             810.0|
|   Columbia|             880.0|
|   Cortland|             562.0|
|   Delaware|             441.0|
|   Dutchess|            2662.0|
|       Erie|            8452.0|
|      Essex|             365.0|
|   Franklin|             366.0|
|     Fulton|             442.0|
|    Genesee|             621.0|
|     Greene|             256.0|
+-----------+------------------+
only showing top 20 rows



In [30]:
# jumlah seluruh fasilitas kesehatan untuk setiap region di NY
changedTypedf.groupBy('Regional Office').count().show()

+--------------------+-----+
|     Regional Office|count|
+--------------------+-----+
|Capital District ...| 3132|
|Western Regional ...| 2758|
|Metropolitan Area...| 3717|
|Western Regional ...| 2424|
|Metropolitan Area...| 3113|
|Metropolitan Area...|10439|
|Central New York ...| 3843|
+--------------------+-----+



In [32]:
# jumlah fasilitas yang tersedia di daerah tertentu
# contohnya Albany
changedTypedf.filter(changedTypedf['County']=='Albany').groupBy('Description').count().show()

+--------------------+-----+
|         Description|count|
+--------------------+-----+
|Mobile Diagnostic...|    1|
|Diagnostic and Tr...|   31|
|School Based Diag...|   36|
|Hospital Extensio...|   38|
|Certified Home He...|   39|
|             Hospice|   32|
|Adult Day Health ...|    2|
|            Hospital|  137|
|Diagnostic and Tr...|   46|
|School Based Hosp...|    1|
|Long Term Home He...|   18|
|Residential Healt...|  165|
+--------------------+-----+

